In [150]:
import numpy as np
import time
np.set_printoptions(suppress = True)

In [132]:
class DiscreteFT():
    
    def __init__(self) -> None:
        """
        For practicipation , 
        will use FFT algorithm in practice 
        
        kernel: 
        e^(-i*(2*pi)*u*x/n) = cos(-2*pi*ux/n) + i*sin(-2*pi*ux/n)
        
        [[cos(-2*pi*ux/n), sin(-2*pi*ux/n)]]
        """
        self.__kernel_image = np.ones((1,1))
        self.__kernel_real = np.zeros((1,1))

    def __makekernel(self, x:np.ndarray):
        
        if x.shape[0] == self.__kernel_image.shape[0]:
            return
        n = x.shape[0]
        ui = np.arange(n).reshape(-1, 1)
        xi = np.arange(n).reshape(1, -1)
        ux = (ui@xi)
        thetas = (-2*np.pi*(ux/n))
        self.__kernel_real = np.cos(thetas)
        self.__kernel_image = np.sin(thetas)
     
    def FT_1d(self, x:np.ndarray, d="real")->np.ndarray:
        
        xt = x.reshape(-1, 1) if x.ndim == 1 else x
        self.__makekernel(xt)
        # ndim == 1 : signal
        # ndim > 1 : complex domain signal
        F = np.hstack(
                ( self.__kernel_real@xt,self.__kernel_image@xt)
            )
        if d == "real":
            return F
        
        elif d == "complex":
            return self.__1d_complex_transform_combination(F)
    
    def __1d_complex_transform_combination(self, x:np.ndarray)->np.ndarray:
        
        """
        F(u) = [[r, i], ... ]

        x = [FT(F*)], 
        
        ## Real part
        kr@F Frf = [a(cos) , b*i*cos] = [a*cos, b*cos]
        
        ## Image part
        ki@F = [a(i*sin) , b*i*(i*sin)] = [-b*sin, a*sin] 
        => Fif = ki@F, exchange columns and Fif[0] *= -1
        [Frf (0,1) | Fif (2,3) ]
        """
        
        x[:, [2,3]] = x[:, [3,2]] 
        x[:, 2] *= -1
        fx = x[:, [0,1]] + x[:, [2,3]]
        return fx
    
    def Inv_1d_FT(self, F_:np.ndarray)->np.ndarray:
        F = F_.copy()
        # conjungate for Frequency
        F[:, 1]*=-1
        f = self.FT_1d(F, d="complex")/F.shape[0] 
        f[:, 1] *= -1
        return f[:, 0]
    
    def FT_2d(self, x:np.ndarray, d="real")->np.ndarray:
        
        F = x.T
        if d == "complex":
            F = x.transpose((1,0,2))
        #y - axis
        F = np.array(list(self.FT_1d(Fi, d) for Fi in F)).transpose((1,0,2))
    
        #x - axis
        F = np.array(list(self.FT_1d(Fi, d) for Fi in F))
        
        if d == "real":
            #complex number combination
            F = np.array(
            list(
                np.hstack(
                    ((Fi[:, 0] - Fi[:, 3]).reshape(-1, 1),
                    (Fi[:, 1] + Fi[:, 2]).reshape(-1, 1))
                ) 
                for Fi in F
            )
            )

        return F
        
    def Inv_2d_FT(self, F_:np.ndarray)->np.ndarray:
        F = F_.copy()
        F[..., 1] *= -1 #conjungate
        f = self.FT_2d(F,d="complex")/(F.shape[0]*F.shape[1])
        f[..., 1] *= -1
        return f[..., 0]
        
ft = DiscreteFT()

In [155]:
x = np.random.randint(low=0, high=255, size=(1920,1080)).astype(np.float64)
t0 = time.time()
F = ft.FT_2d(x)
t1 = time.time()
xh = ft.Inv_2d_FT(F)
t2 = time.time()
print(np.sum((xh-x)**2))
print(f"F:{t1-t0}, F^-1:{t2-t1}")

2.6146684818388965e-15
F:10.465530157089233, F^-1:26.445576429367065
